In [3]:
'''
Aryaman Pandya 
Sequential Machine Learning 
Building a Vanilla RNN 
Model and trainer implementation 
Following https://github.com/rasbt/deeplearning-models/blob/master/pytorch_ipynb/rnn/rnn_bi_multilayer_lstm_own_csv_agnews.ipynb
implementation minus the memory unit for now 
'''
import torch 
import pandas as pd
import numpy as np
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import plotly
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.utils.data import Dataset, DataLoader

#Class definition of Vanilla RNN 
class VanillaRNN(nn.Module): 
    
    def __init__(self, vocab_size, embed_size, input_len, hidden_size, output_len, num_layers) -> None:
        super(VanillaRNN, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.hidden_size = hidden_size 
        self.input_len = input_len 
        self.output_len = output_len 
        self.rnn = nn.RNN(nput_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, dropout=0.5,
                                batch_first=True, bidirectional=True) #graph module to compute next hidden state 
        
        self.hidden2label = nn.Linear(2*hidden_size, 2)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropoutLayer = nn.Dropout()

    def forward(self, x, text_len):
        embedded = self.encoder(x)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_len)
        output, hidden = self.rnn(packed_embedded)  # Pass the initial hidden state 'h' to the RNN
        
        
        # Flatten the output tensor to match the linear layer input size
        output = output.contiguous().view(-1, 2 * self.hidden_size)
        
        # Apply dropout to the flattened output
        output = self.dropoutLayer(output)
        
        # Pass the output through the linear layer
        output = self.hidden2label(output)
        
        # Apply softmax activation to get probabilities
        output = self.softmax(output)
        
        return output, hidden

    def init_h(self):
        return torch.zeros(1, self.len_h)

In [4]:
df = pd.read_csv('ag_news_csv/train.csv', header=None, index_col=None)

In [12]:
df.columns = ['label', 'title', 'data']
df['label'] = df['label']-1
df.head()

,label,title,data
0,2,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,2,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,2,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,2,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [14]:
df.to_csv('train_set.csv')

In [18]:
df_test = pd.read_csv('ag_news_csv/test.csv', header=None, index_col=None)
df_test.columns = ['label', 'title', 'data']
df_test['label'] = df_test['label']-1
df_test.to_csv('test_set.csv')